<a href="https://colab.research.google.com/github/Giymo11/cryptobees/blob/master/cryptotaxes2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
# This installs our necessary dependencies
!pip install google-colab
!pip install pandas 
!pip install python-dateutil
!pip install pygsheets

In [60]:
import logging
import sys
import pprint
import datetime
import dateutil
import pygsheets

# to save some typing
import pandas as pd

from dateutil.parser import parse

In [61]:
from google.colab import drive
import os

mount_point = '/content/gdrive'
drive.mount(mount_point)

gsheets = pygsheets.authorize(service_file='/content/gdrive/My Drive/cryptotaxes/cryptotaxes-0-1f774cf735ef.json')
year = 2020

# The google drive folder you want your data saved in
dir_name="/My Drive/cryptotaxes/crypto" + str(year)

directory = mount_point + dir_name
if not os.path.exists(directory):
  os.makedirs(directory)

log_filename = directory + '/cryptotaxes.log'


# Create a custom logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create handlers
console_handler = logging.StreamHandler()
file_handler = logging.FileHandler(log_filename)
console_handler.setLevel(logging.INFO)
file_handler.setLevel(logging.INFO)

# Create formatters and add it to handlers
console_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
file_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(console_format)
file_handler.setFormatter(file_format)

# Add handlers to the logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)




kraken_csv_name = directory + '/kraken_2020.csv'
kraken_df = pd.read_csv(kraken_csv_name)

gsheet_name = 'cryptotaxes2020'
# Don't forget to share your spreadsheet with this script!
sh = gsheets.open(gsheet_name)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [68]:


def read_pricedata(coin_name):
  csv_name = directory + '/../historical_data/' + coin_name + '-eur-max.csv'
  pricedata_df = pd.read_csv(csv_name)

  index = pricedata_df['snapped_at'].apply(lambda x: parse(x).date().isoformat())
  price = pricedata_df['price']
  return dict(zip(index, price))

coins = ['btc', 'eth', 'atom', 'dot', 'xtz']

historical_data = dict(zip(coins, map(read_pricedata, coins)))


translations = {
  'XETH': 'ETH',
  'XXMR': 'XMR',
  'XXBT': 'BTC',
  'XXRP': 'XRP',
  'XZEC': 'ZEC',
  'XMLN': 'MLN',
  'XXLM': 'XLM',
  'REPV2': 'REP',
  'ZEUR': 'EUR'
}

kraken_df['asset'] = kraken_df['asset'].apply(lambda x: x if not x in translations else translations[x])
kraken_df['asset'] = kraken_df['asset'].apply(lambda x: x if not '.S' in x else x.replace('.S', '') + '.Z!')


In [72]:


list_transfers = []
list_trades = []
list_earnings = []

def append_to_list(my_list, wallet, txtype, timestamp, tx_id, coin_id, shares, counterparty):
  list_entry = {
      'wallet': wallet, 
      'txtype': txtype, 
      'timestamp': timestamp,
      'tx_id': tx_id, 
      'coin_id': coin_id, 
      'shares': shares, 
      'counterparty': counterparty
  }
  my_list.append(list_entry)


def kraken_add_row_to_list(my_list, row, wallet = 'kraken', counterparty = ''):
  append_to_list(my_list,
      wallet = wallet,
      txtype = row['type'],
      timestamp = parse(row['time']).date().isoformat(),
      tx_id = row['refid'],
      coin_id = row['asset'],
      shares = row['amount'],
      counterparty = counterparty
  )
  if row['fee'] != 0:
    append_to_list(my_list,
        wallet = wallet,
        txtype = row['type'] + '.fee',
        timestamp = parse(row['time']).date().isoformat(),
        tx_id = row['refid'],
        coin_id = row['asset'],
        shares = -1 * row['fee'],
        counterparty = wallet
    )

def convert_kraken(row):
  if row['type'] == 'deposit' and type(row['txid']) == str:
    kraken_add_row_to_list(list_transfers, row, wallet = 'kraken', counterparty='unknown')
  elif row['type'] == 'withdrawal' and type(row['txid']) == str:
    kraken_add_row_to_list(list_transfers, row, wallet = 'kraken', counterparty='unknown')
  elif row['type'] == 'trade':
    kraken_add_row_to_list(list_trades, row, wallet = 'kraken', counterparty='kraken')
  elif row['type'] == 'transfer' and row['subtype'] == 'spottostaking':
    kraken_add_row_to_list(list_transfers, row, wallet = 'kraken', counterparty = 'kraken.Z!')
  elif row['type'] == 'transfer' and row['subtype'] == 'stakingfromspot':
    kraken_add_row_to_list(list_transfers, row, wallet = 'kraken.Z!', counterparty = 'kraken')
  elif row['type'] == 'staking':
    kraken_add_row_to_list(list_earnings, row, wallet = 'kraken.Z!', counterparty = 'kraken.Z!')


kraken_df.apply(convert_kraken, axis=1)

df_transfers = pd.DataFrame(list_transfers)
df_trades = pd.DataFrame(list_trades)
df_earnings = pd.DataFrame(list_earnings)

worksheet = sh[0]
worksheet.clear()

position = (1,1)
worksheet.cell(position).set_text_format('bold', True).set_value('Transfers')
position = (position[0] + 1, position[1])
worksheet.set_dataframe(df_transfers, position)

position = (1, position[1] + df_transfers.shape[1] + 1)
worksheet.cell(position).set_text_format('bold', True).set_value('Trades')
position = (position[0] + 1, position[1])
worksheet.set_dataframe(df_trades, position)

position = (1, position[1] + df_trades.shape[1] + 1)
worksheet.cell(position).set_text_format('bold', True).set_value('Earnings')
position = (position[0] + 1, position[1])
worksheet.set_dataframe(df_earnings, position)


In [78]:

df_tradegroups = df_trades.groupby('tx_id')

list_trades_unresolved = []
list_transfers_unresolved = []
list_tranches_trades = []


def get_price(coin_id, timestamp):
  coin = coin_id.replace('.Z!', '').lower()
  pricedata = historical_data.get(coin)
  if pricedata is not None:
    return pricedata.get(row['timestamp'])

def add_value_eur_to_row(row):
  value = get_price(row['coin_id'], row['timestamp'])
  if value is not None:
    value = value * row['shares']
  new_row = dict(row)
  new_row['value_eur'] = value
  return new_row


# loc 0 is sell, loc 1 is fee, loc 2 is buy
def add_tradegroup_to_list(trade_list, key, group):
  sell_entry = add_value_eur_to_row(group.iloc[0])
  trade_list.append(sell_entry)
  fee_entry = add_value_eur_to_row(group.iloc[1])
  trade_list.append(fee_entry)
  buy_entry = {
      'wallet': group['wallet'].iloc[2],
      'coin_id' : group['coin_id'].iloc[2],
      'timestamp': group['timestamp'].iloc[2],
      'shares': group['shares'].iloc[2],
      'value_eur': -1 * sell_entry['value_eur'],
      'fee_eur': -1 * fee_entry['value_eur'],
      'tx_id': key,
      'acquired': 'trading'
  }
  list_tranches_trades.append(buy_entry)


for key, group in df_tradegroups:
  # if it's a sell
  if group['coin_id'].iloc[0] == 'EUR' and group['shares'].iloc[0] > 0:
    add_tradegroup_to_list(list_trades_unresolved, key, group)
  # not a buy
  elif not (group['coin_id'].iloc[0] == 'EUR' and group['shares'].iloc[0] < 0):
    add_tradegroup_to_list(list_trades_unresolved, key, group)


for row in list_transfers:
  if row['txtype'].endswith('.fee'):
    trade_entry = add_value_eur_to_row(row)
    list_trades_unresolved.append(trade_entry)
    transfer_entry = dict(row)
    transfer_entry['coin_id'] = 'EUR'
    transfer_entry['shares'] = trade_entry['value_eur']
    list_transfers_unresolved.append(transfer_entry)
  else:
    list_transfers_unresolved.append(row)


df_trades_unresolved = pd.DataFrame(list_trades_unresolved)
df_transfers_unresolved = pd.DataFrame(list_transfers_unresolved)


worksheet = sh[1]
worksheet.clear()

position = (1,1)
worksheet.cell(position).set_text_format('bold', True).set_value('Unresolved Trades')
position = (position[0] + 1, position[1])
worksheet.set_dataframe(df_trades_unresolved, position)

position = (position[0] + df_trades_unresolved.shape[0] + 3, position[1])
worksheet.cell(position).set_text_format('bold', True).set_value('Unresolved Transfers')
position = (position[0] + 1, position[1])
worksheet.set_dataframe(df_transfers_unresolved, position)



getting price of eth on 2020-12-28 is 612.0952116149559
getting price of eth on 2020-12-28 is 612.0952116149559
getting price of eth on 2020-12-28 is 612.0952116149559
getting price of eth on 2020-12-28 is 612.0952116149559
getting price of eth on 2020-08-30 is 335.42549375172285
getting price of eth on 2020-10-21 is 311.3576659295714
getting price of btc on 2020-10-27 is 11059.1474408289
getting price of btc on 2020-12-28 is 21651.45790437261


In [80]:

list_tranches_buys = []
for key, group in df_tradegroups:
  # if it's a buy
  if group['coin_id'].iloc[0] == 'EUR' and group['shares'].iloc[0] < 0:
    entry = {
        'wallet': group['wallet'].iloc[2],
        'coin_id' : group['coin_id'].iloc[2],
        'timestamp': group['timestamp'].iloc[2],
        'shares': group['shares'].iloc[2],
        'value_eur': -1 * group['shares'].iloc[0],
        'fee_eur': -1 * group['shares'].iloc[1],
        'tx_id': key,
        'acquired': 'buying'
    }
    list_tranches_buys.append(entry)

list_tranches_earnings = []
for row in list_earnings:
  value = get_price(row['coin_id'], row['timestamp'])
  if value is not None:
    value = value * row['shares']
  entry = {
      'wallet': row['wallet'],
      'coin_id' : row['coin_id'],
      'timestamp': row['timestamp'],
      'shares': row['shares'],
      'value_eur': value,
      'fee_eur': 0,
      'tx_id': row['tx_id'],
      'acquired': 'staking'
  }
  list_tranches_earnings.append(entry)

list_tranches = list_tranches_buys + list_tranches_earnings + list_tranches_trades
df_tranches = pd.DataFrame(list_tranches).sort_values(by=['coin_id', 'timestamp'], ascending=True)

sum_staking_earnings = df_tranches['value_eur'].where(df_tranches['acquired'] == 'staking').sum()

worksheet = sh[2]
worksheet.clear()

position = (1,1)
worksheet.cell(position).set_text_format('bold', True).set_value('Tranches')
position = (position[0] + 1, position[1])
worksheet.set_dataframe(df_tranches, position)

position = (1, position[1] + 1 + df_tranches.shape[1])
worksheet.cell(position).set_text_format('bold', True).set_value('Earnings Total')
position = (position[0] + 1, position[1])
worksheet.cell(position).set_value(sum_staking_earnings)


getting price of atom on 2020-09-23 is 3.595577117340638
getting price of xtz on 2020-09-24 is 1.6728113110911214
getting price of atom on 2020-09-27 is 3.9568310713983967
getting price of xtz on 2020-09-28 is 1.8710072164687304
getting price of atom on 2020-09-30 is 4.4889863993309405
getting price of xtz on 2020-10-01 is 1.875224399329488
getting price of atom on 2020-10-04 is 4.28172196431842
getting price of xtz on 2020-10-05 is 1.8052521633502372
getting price of atom on 2020-10-07 is 4.042838431648615
getting price of xtz on 2020-10-08 is 1.7548500145600836
getting price of atom on 2020-10-11 is 4.66391970358942
getting price of xtz on 2020-10-12 is 1.9034333884122396
getting price of atom on 2020-10-14 is 5.119816629072986
getting price of xtz on 2020-10-15 is 1.9464627775398664
getting price of atom on 2020-10-18 is 4.6669751552116665
getting price of xtz on 2020-10-19 is 1.9018064502148688
getting price of dot on 2020-10-20 is 3.391800812186592
getting price of atom on 2020-10

<Cell J2 1.9604214059389957>